<a href="https://colab.research.google.com/github/suheon927/PerceptronPractice/blob/main/Perceptron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import math
import random

x = [[0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0], [0, 0, 1, 1], [0, 1, 0, 0], [0, 1, 0, 1], [0, 1, 1, 0], [0, 1, 1, 1],
      [1, 0, 0, 0], [1, 0, 0, 1], [1, 0, 1, 0], [1, 0, 1, 1], [1, 1, 0, 0], [1, 1, 0, 1], [1, 1, 1, 0], [1, 1, 1, 1]]
y = [0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0]

# 은닉층을 위한 16개의 가중치 설정
weights = [[random.uniform(-0.5, 0.5) for _ in range(4)] for _ in range(16)]
output_weights = [random.uniform(-0.5, 0.5) for _ in range(16)]
bias = [random.uniform(-0.5, 0.5) for _ in range(16)]

l_rate = 0.0007  # 학습률을 더 낮춤
n_epoch = 10000  # 에포크 수를 10000으로 늘림

def sigmoid(x):
    return 1 / (1 + math.exp(-x))

# 은닉층 출력 계산 함수
def calculate(input):
    global weights, bias
    activations = []
    for i in range(16): # 16개의 뉴런에 대해서
        activation = bias[i]
        for j in range(4):
          activation += weights[i][j] * input[j]
        activations.append(sigmoid(activation))
    return activations

# 학습 알고리즘
def train_weights(x, y, l_rate, n_epoch):
    global weights, output_weights, bias
    for epoch in range(n_epoch):
        sum_error = 0.0
        for row, target in zip(x, y):
            # 1. 은닉층 출력 계산 (output1, output2 반환)
            outputs = calculate(row)

            # 2. 출력층 오차 계산
            final_output = sum([outputs[i] * output_weights[i] for i in range(16)])  # 은닉층 출력의 합
            error = target - final_output

            # 3. 출력 가중치 업데이트
            for i in range(16):
                output_weights[i] += l_rate * error * outputs[i]

            # 4. 은닉층 오차 계산 및 가중치 업데이트
            for i in range(16):
                hidden_error = output_weights[i] * error
                for j in range(4):
                    weights[i][j] += l_rate * hidden_error * row[j]


            # 5. 바이어스 업데이트
            for i in range(16):
                bias[i] += l_rate * error

            sum_error += error**2

        print(f'Epoch={epoch}, l_rate={l_rate}, sum_error={sum_error}')
    return weights

# 학습 실행
train_weights(x, y, l_rate, n_epoch)

# XOR 문제 예측 결과 확인
print("\nXOR 문제 예측 결과:")
for row in x:
    outputs = calculate(row)
    final_output = sum([outputs[i] * output_weights[i] for i in range(16)])
    prediction = 1.0 if final_output >= 0.5 else 0.0
    print(f"입력: {row}, 예측: {prediction}")

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Epoch=5018, l_rate=0.0007, sum_error=4.016700050655858
Epoch=5019, l_rate=0.0007, sum_error=4.016700053835967
Epoch=5020, l_rate=0.0007, sum_error=4.016700057022311
Epoch=5021, l_rate=0.0007, sum_error=4.016700060214877
Epoch=5022, l_rate=0.0007, sum_error=4.016700063413649
Epoch=5023, l_rate=0.0007, sum_error=4.016700066618613
Epoch=5024, l_rate=0.0007, sum_error=4.016700069829755
Epoch=5025, l_rate=0.0007, sum_error=4.016700073047059
Epoch=5026, l_rate=0.0007, sum_error=4.01670007627051
Epoch=5027, l_rate=0.0007, sum_error=4.0167000795000956
Epoch=5028, l_rate=0.0007, sum_error=4.0167000827358
Epoch=5029, l_rate=0.0007, sum_error=4.016700085977609
Epoch=5030, l_rate=0.0007, sum_error=4.016700089225507
Epoch=5031, l_rate=0.0007, sum_error=4.016700092479481
Epoch=5032, l_rate=0.0007, sum_error=4.016700095739513
Epoch=5033, l_rate=0.0007, sum_error=4.016700099005596
Epoch=5034, l_rate=0.0007, sum_error=4.016700102277707
Epoch=5035, l_rate=0.0007, sum_